In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up paths
base_dir = "C:/Photos"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 20% for validation

# Load training data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',  # 'binary' because it's a 2-class problem (GO/NG)
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the trained model
model.save("fan_model.h5")


Found 270 images belonging to 2 classes.
Found 66 images belonging to 2 classes.


c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 375ms/step - accuracy: 0.5817 - loss: 1.1719 - val_accuracy: 0.6212 - val_loss: 0.6680
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 246ms/step - accuracy: 0.5750 - loss: 0.6536 - val_accuracy: 0.6212 - val_loss: 0.6697
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.6674 - loss: 0.5793 - val_accuracy: 0.7576 - val_loss: 0.6445
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - accuracy: 0.7967 - loss: 0.4616 - val_accuracy: 0.8333 - val_loss: 0.5898
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.8910 - loss: 0.3138 - val_accuracy: 0.7879 - val_loss: 0.5362
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - accuracy: 0.9585 - loss: 0.1673 - val_accuracy: 0.8030 - val_loss: 0.5211
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 245ms/step - accuracy: 0.9694 - loss: 0.0954 - val_accuracy: 0.5000 - val_loss: 0.6807
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 249ms/step - accuracy: 0.9947 - loss: 0.0395 - val_accuracy: 0.8788 - val_loss:

In [11]:
import cv2
import numpy as np
import tensorflow as tf
import glob
import os

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    prediction = model.predict(img)
    
    # Handle cases where the prediction might not be well-formed
    if prediction is not None and len(prediction) > 0:
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold for binary classification
        confidence = prediction[0][0] * 100  # Confidence score in percentage
        return class_idx, confidence
    else:
        return None, None

# Function to classify all images in given directories
def classify_images_in_directories(directories):
    for directory_path in directories:
        print(f"\nClassifying images in directory: {directory_path}")
        
        # Use glob to find all image files (supports .jpg, .png, .jpeg)
        image_files = glob.glob(os.path.join(directory_path, "*.jpg")) + \
                      glob.glob(os.path.join(directory_path, "*.png")) + \
                      glob.glob(os.path.join(directory_path, "*.jpeg"))
        
        if not image_files:
            print("No images found in the directory!")
            continue
        
        for image_path in image_files:
            result, confidence = classify_image(image_path)
            if result is not None:
                if result == 1:
                    print(f"GO (Correctly Aligned) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                else:
                    print(f"NG (Defective) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
            else:
                print(f"Unable to classify image: {os.path.basename(image_path)}")

# Example usage: Classify images from both "60" and "100" folders
directories = ["C:/Photos/60", "C:/Photos/100"]
classify_images_in_directories(directories)



Classifying images in directory: C:/Photos/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
GO (Correctly Aligned) with 74.96% confidence for image: image-30-B20008.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
GO (Correctly Aligned) with 99.53% confidence for image: image-30-B20009.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
GO (Correctly Aligned) with 98.90% confidence for image: image-30-B20023.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
GO (Correctly Aligned) with 98.21% confidence for image: image-30-B20024.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
GO (Correctly Aligned) with 98.72% confidence for image: image-30-B20025.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
GO (Correctly Aligned) with 98.16% confidence for image: image-30-B20026.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
GO (Correctly Aligned) with 95.80% confidence for image: image-30-B20027.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
GO (Correctly Aligned) with 95.07% confidence for image: image-30-B20028.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/

### The code needs to import classify function

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import gradio as gr
from PIL import Image

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image):
    # Convert the image to a format suitable for OpenCV
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    
    # Make prediction
    prediction = model.predict(img)
    
    # Determine class index and confidence
    if prediction is not None and len(prediction) > 0:
        class_idx = 1 if prediction[0] >= 0.9 else 0  # Threshold for binary classification
        confidence = prediction[0][0] * 100  # Confidence score in percentage
        
        if class_idx == 1:
            return f"GO (Correctly Aligned) with {confidence:.2f}% confidence"
        else:
            return f"NG (Defective) with {confidence:.2f}% confidence"
    else:
        return "Unable to classify image"

# Create Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Textbox(label="Classification Result"),
    title="Cooling Fan GO/NG Inspection",
    description="Upload an image of a cooling fan to check if it is correctly aligned (GO) or defective (NG).",
)

# Launch the interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
